In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import re
import pandas as pd
import numpy as np

import drive_access
import parsemadzu

re_lcd = re.compile(r"(.*)\.lcd$")
re_txt = re.compile(r"(.*)\.txt$")

files = drive_access.list_files()
print(len(files))

849


In [3]:
def get_path(file: dict) -> str:
    """Find path to file, i.e. its folder location."""
    current_path = ""
    current_file = file
    for i in range(5):
        parents = current_file.get("parents", [])
        parent = parents[0] if parents else None
        
        if parent is None:
            break
        
        parent_search = [f for f in files if f["id"] == parent]
        
        if parent_search is None or len(parent_search) == 0:
            break
        
        parent_file = parent_search[0]
        current_path = os.path.join(parent_file["name"], current_path)
        current_file = parent_file
        
    return current_path

get_path(files[3])

'Project1/2021/09_September/20210901/'

In [4]:
#[f for f in files if "1-LVxSt2qo-5ioi3er3t3QsatEPo9V3-C" in f.get("parents", [])]

rows = []
columns = ["name", "drive_id", "path", "txt_drive_id"]

lcd_files = [f for f in files if re_lcd.match(f["name"])]

for lcd_file in lcd_files:
    txt_file_name = os.path.splitext(lcd_file["name"])[0] + ".txt"
    
    # Find files in same folder with same name but .txt extension instead of .lcd
    txt_file_search = [f for f in files if f["name"] == txt_file_name and f["parents"] == lcd_file["parents"]]

    txt_file = None
    if txt_file_search:
        txt_file = txt_file_search[0]
        
    rows.append([
        lcd_file["name"],
        lcd_file["id"],
        get_path(lcd_file),
        txt_file["id"] if txt_file else np.nan
    ])
    
df = pd.DataFrame(rows, columns=columns).sort_values("name")
df_has_txt = df[np.logical_not(pd.isna(df["txt_drive_id"]))]
df_has_txt

,name,drive_id,path,txt_drive_id
258,20210901 AA GL_01092021_001.lcd,1_PWDxSNzYxkXnHlbxCqxHXibZ59lpSUS,Project1/2021/09_September/20210901/,1p5ZTxdqlJH71DdajNhF83a0SvNRZx4zr
257,20210901 AA GL_01092021_002.lcd,1aj6ei7pTwTZjFw8PNJuNrpuN4HcGFWOe,Project1/2021/09_September/20210901/,1p3PV-Gcot62ARGqNYnyhdw_inTEywlhn
256,20210901 AA GL_01092021_003.lcd,1_qAbDP05iBSZCk6VQukWOkXeOCl1Je_Z,Project1/2021/09_September/20210901/,1p36wr3YTR388msuxEMCQ8esxetZWxUWI
255,20210901 AA GL_01092021_004.lcd,1au8huAEJM40gPUQm52Ll2ze0WuaAPD66,Project1/2021/09_September/20210901/,1pUemLBv7O2ZInvDysN0klCO24MbFB_2n
254,20210901 AA GL_01092021_005.lcd,1a7VS2kPXJayYGQlDu0AHUImL02fDj88a,Project1/2021/09_September/20210901/,1pX9zGM1EHOTMe5xv90KyKskYQ-X2UZKr
...,...,...,...,...
4,20210930 VT_30092021_006.lcd,1Fndh4fymWutFLxQgSFvY94sHJM9vjFt_,Project1/2021/09_September/20210930/,1bgfr8ZObfnACqEM-OnZRn2s1zLinUNUm
3,20210930 VT_30092021_007.lcd,1Frc20sGo7D6J0-pe_k-AWQBmPFe-9IUb,Project1/2021/09_September/20210930/,1bj55lIkH8cXdzFiEscO0rXLHqFRnVvZK
2,20210930 VT_30092021_008.lcd,1GDUsiVC3gNOrP32T1ISyyzafFWjTJ-y2,Project1/2021/09_September/20210930/,1bZKU9LfpeSR3hl-yCBXuzJRLxgwwsf_J
1,20210930 VT_30092021_009.lcd,1GL52WJA05bW-W-az0rx_4iXrKgCbZ5K_,Project1/2021/09_September/20210930/,1bQs_o7r90bHunLq0wQrxURg-Pzqefsrb


In [5]:
def combine_compound_concentrations(df_txt_files: pd.DataFrame) -> pd.DataFrame:
    """Combine a list of result files into a table of concentrations of compounds detected in each experiment."""
    
    compound_names = None
    rows = []

    for _, row_in in df_txt_files.iterrows():
        f = drive_access.get_file(row_in["txt_drive_id"])
        
        sections = parsemadzu.parse_sections(f)
        compounds = parsemadzu.get_compound_table(sections, detector="B")

        if compound_names is None:
            compound_names = compounds["Name"]
            
        if compounds is not None and compound_names.equals(compounds["Name"]):
            concentrations = compounds["Conc."]
        else:
            if compounds is None:
                print(f"Skipping {row_in['txt_drive_id']} because of empty compound list")
            else:
                print(f"Skipping {row_in['txt_drive_id']} because of different compound list")
                print(compounds["Name"].tolist())
            concentrations = np.empty(shape=compound_names.shape)
            concentrations[:] = np.nan

        
        rows.append(concentrations.tolist())

    colnames = compound_names.tolist()

    df_concs = pd.DataFrame(rows, columns=colnames)
    df_concs["txt_drive_id"] = df_txt_files["txt_drive_id"].to_numpy()
    
    return df_concs

df_compounds_combined = combine_compound_concentrations(df_has_txt)

Skipping 1ofGI7RTutyVTUwIYRKjds_qqep9QUjI7 because of different compound list
['Acetic Acid', 'Glucose', 'Glycerol', 'Ethanol', 'Xylose', 'Citric Acid']
Skipping 1od8lAvzvPdWAWWsS0ysqw8nt3S_L5J9s because of different compound list
['Acetic Acid', 'Glucose', 'Glycerol', 'Ethanol', 'Xylose', 'Citric Acid']
Skipping 1oTp-MRXok9_UH_GOUVqMmYKpjO0eADIB because of different compound list
['Acetic Acid', 'Glucose', 'Glycerol', 'Ethanol', 'Xylose', 'Citric Acid']
Skipping 1obLIKHhH63L7W26DvFskfMmT_xznUlKh because of different compound list
['Acetic Acid', 'Glucose', 'Glycerol', 'Ethanol', 'Xylose', 'Citric Acid']
Skipping 1oi9F6Zxy3_ZgAOCGXkGbGiPLBlJE-he9 because of different compound list
['Acetic Acid', 'Glucose', 'Glycerol', 'Ethanol', 'Xylose', 'Citric Acid']
Skipping 1oOCuocfnUTXXBM-7ZiHip63AcyZ1uSPS because of empty compound list
Skipping 1oCPuEVyc8NrxWMXHHr684LJ7wAfgCjRD because of empty compound list
Skipping 1oHUiXvNK_cQMYsh2pdZkemEeKayLdh4l because of empty compound list
Skipping 1nwp

In [6]:
df_upload = df_has_txt.merge(df_compounds_combined, on="txt_drive_id")

## Save to google sheet

In [7]:
import gspread
import df2gspread.df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials


spreadsheet_id = "14DSZKVv0xCJFgxJUDYk-hgTrLtFCkWhZLsexDM3G8L0"

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
cred = ServiceAccountCredentials.from_json_keyfile_name(
            "secrets/biolabautomation-d5dc83b8ddf7.json", scope)
gc = gspread.authorize(cred)
d2g.upload(df_upload, spreadsheet_id, "autoimport", credentials=cred, row_names=False)

<Worksheet 'autoimport' id:0>